In [1]:
# import section

import pandas as pd
import pickle
import shutil
import os
from sklearn.model_selection import train_test_split
from pyfume import Clustering
from pyfume import AntecedentEstimator
from pyfume import FireStrengthCalculator
from pyfume import SugenoFISBuilder
from pyfume import ConsequentEstimator

In [2]:
# load best hyperparameters from pkl file into pandas dataframe

best_hyperparameters = pd.read_pickle('model_training/best_hyperparameters.pkl')

# Extract the first row from the 'params' column
params_dict = best_hyperparameters['params'].iloc[0] 

# Verify the type to ensure it's already a dictionary
if isinstance(params_dict, dict):
    # Dynamically create variables with names corresponding to the dictionary keys
    for key, value in params_dict.items():
        locals()[key] = value

    # Print the variables to verify
    for key in params_dict.keys():
        print(f"{key}: {locals()[key]}")
else:
    print("Error: The 'params' column does not contain a dictionary.")

    # Base folder for .pkl files
base_folder = 'feature_selection'

# Construct the full path
pkl_file_path = f'{base_folder}/{dataframe_name}.pkl'

# Load the dataframe and assign it to train_df
train_df = pd.read_pickle(pkl_file_path)

# Print to verify
print(f"Loaded dataframe from: {pkl_file_path}")

sample_size: 605
random_state: 2024
nr_clus: 13
mf_shape: trapmf
merge_threshold: 1.0
m: 2.0
dataframe_name: train_df_all_features_10
covariance_type: tied
consequent_method: global_LSE
clustering_method: gmm
Loaded dataframe from: feature_selection/train_df_all_features_10.pkl


In [3]:
# Assign the train dataset to df
df = train_df

# Split the data into features and target
X = df.drop(columns=['target'])
y = df['target']

X = X.to_numpy()
y = y.to_numpy()

# Variable names for future steps
variable_names = df.columns[:-1]

In [4]:
# Create a Clusterer object and run clustering
cl = Clustering.Clusterer(x_train=X, y_train=y, nr_clus=nr_clus)

# Adjust the clustering method based on the parameters
if clustering_method in ['fcm', 'fcm_binary', 'fst-pso', 'gk']:
    # Pass the fuzziness coefficient m for these methods
    cluster_centers, partition_matrix, _ = cl.cluster(method=clustering_method, m=m)
elif clustering_method == 'gmm':
    # Pass covariance_type for GMM
    cluster_centers, partition_matrix, _ = cl.cluster(method=clustering_method, covariance_type=covariance_type)
else:
    # For other methods, run the clustering normally
    cluster_centers, partition_matrix, _ = cl.cluster(method=clustering_method)

In [5]:
# Estimate membership functions
ae = AntecedentEstimator(x_train=X, partition_matrix=partition_matrix)
antecedent_parameters = ae.determineMF(mf_shape=mf_shape, merge_threshold=merge_threshold)

/Users/peter/.pyenv/versions/FRP31110/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [6]:
# Calculate firing strengths
fsc = FireStrengthCalculator(antecedent_parameters=antecedent_parameters, nr_clus=nr_clus, variable_names=variable_names)
firing_strengths = fsc.calculate_fire_strength(data=X)

Index(['p728_i0_Category_B', 'p1200_i0_Category_E', 'p10895_i0_Category_E',
       'p6222_i0', 'p4041_i0_Category_B', 'p6148_i0_Category_E',
       'p3506_i0_Category_E', 'p1418_i0_Category_B', 'p4295_i0_Category_E',
       'p1598_i0_Category_C'],
      dtype='object')


In [7]:
# Estimate consequent parameters
ce = ConsequentEstimator(x_train=X, y_train=y, firing_strengths=firing_strengths)
consequent_parameters = ce.zero_order(method=consequent_method)

# Build the Sugeno FIS model
simpbuilder = SugenoFISBuilder(
    antecedent_sets=antecedent_parameters,
    consequent_parameters=consequent_parameters,
    variable_names=variable_names,
    model_order='zero',
    extreme_values=None,
    save_simpful_code=True
)
model = simpbuilder.get_model()

 * Detected 13 rules / clusters
 * Code saved to file Simpful_code.py
 * Detected Sugeno model type


In [8]:
# Save the model to a file
with open('model_training/final_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [9]:
# move Simpful code to model_training folder

# Define the source and destination paths
source_file = 'Simpful_code.py'
destination_folder = 'model_training'

# Ensure the destination folder exists, create it if it doesn't
os.makedirs(destination_folder, exist_ok=True)

# Move the file
shutil.move(source_file, os.path.join(destination_folder, source_file))

print(f"{source_file} has been moved to {destination_folder}.")

Simpful_code.py has been moved to model_training.
